In [1]:
# Run this cell to produce the output CSV file for the visualization!

import numpy as np
import sklearn as skl
import six
import tensorflow as tf

from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.inprocessing import AdversarialDebiasing

from fair_selection_aif import AIF360Search, DEFAULT_ADB_PARAMS

import os


dataset = AdultDataset()
models = {'AdversarialDebiasing': AdversarialDebiasing}
metrics = {'ClassificationMetric': [ClassificationMetric,
               'num_generalized_true_positives',
               'num_true_negatives',
               'false_positive_rate',
               'false_negative_rate',
               'num_pred_positives'
           ],
           'BinaryLabelDatasetMetric': [BinaryLabelDatasetMetric, 'disparate_impact']
          }

unprivileged = [{'race': 0}]
privileged = [{'race': 1}]

# Hyperparameters may either be specified as a dictionary of string to lists, or by an empty dictionary to
# use the default ones set by sklearn (or AIF360). The keys are the names of the hyperparameters, and the
# values and lists of possible values to form a grid search over

# For the AdversarialDebiasing classifier, there are a bunch of parameters that AIF360 expects to be in the constructor.
# Here we are using a function that takes in the unprivileged and privileged groups, and outputs a dictionary containing
# the (default) expected parameters for the AdversarialDebiasing class
parameters = {
              'AdversarialDebiasing': DEFAULT_ADB_PARAMS(unprivileged=unprivileged, privileged=privileged)
             }
thresholds = [i * 10.0/100 for i in range(5)]

Search = AIF360Search(models, metrics, parameters, thresholds)
Search.grid_search(dataset, privileged=privileged, unprivileged=unprivileged)

Search.to_csv("interface/static/data/test-file.csv")

/Users/Jesse/miniconda3/envs/python36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


epoch 0; iter: 0; batch classifier loss: 16.565380; batch adversarial loss: 0.566661
epoch 0; iter: 200; batch classifier loss: 7.461178; batch adversarial loss: 0.615761
epoch 1; iter: 0; batch classifier loss: 2.289180; batch adversarial loss: 0.497734
epoch 1; iter: 200; batch classifier loss: 10.202908; batch adversarial loss: 0.534607
epoch 2; iter: 0; batch classifier loss: 1.532035; batch adversarial loss: 0.534189
epoch 2; iter: 200; batch classifier loss: 1.844865; batch adversarial loss: 0.484116
epoch 3; iter: 0; batch classifier loss: 1.176221; batch adversarial loss: 0.503239
epoch 3; iter: 200; batch classifier loss: 0.683210; batch adversarial loss: 0.430215
epoch 4; iter: 0; batch classifier loss: 1.469453; batch adversarial loss: 0.418813
epoch 4; iter: 200; batch classifier loss: 0.486674; batch adversarial loss: 0.452138
epoch 5; iter: 0; batch classifier loss: 0.926523; batch adversarial loss: 0.507113
epoch 5; iter: 200; batch classifier loss: 0.841391; batch adver

epoch 48; iter: 200; batch classifier loss: 0.339569; batch adversarial loss: 0.392679
epoch 49; iter: 0; batch classifier loss: 0.301715; batch adversarial loss: 0.406505
epoch 49; iter: 200; batch classifier loss: 0.468309; batch adversarial loss: 0.485221


In [2]:
from bokeh.io import output_notebook
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [3]:
from interface.plot import *

# Function that takes in a document and attaches the bokeh server to it
def modify_doc(doc):
    
    # Load custom styles (for notebook only)
    custom_css = Div(text="<link rel='stylesheet' type='text/css' href='interface/static/css/styles-notebook.css'>")
    add_btn = Button(label="Add Plot", button_type="success")
    remove_btn = Button(label="Remove Plot", button_type="danger")

    # Construct our viewport
    l = layout([
        [custom_css],
        create_plot("interface/static/data/test-file.csv")
    ], sizing_mode="fixed", css_classes=["layout-container"])

    doc.add_root(l)

# Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler)

# Show it in the notebook!
show(app)